In [ ]:
# Connect to drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pyspark

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/cmdoptions.py", line 21, in <module>
    from pip._vendor.packaging.utils import canonicalize_name
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/__init__.py", line 10, in <module>
    import glob
  File "/usr/lib/python3.7/glob.py", line 5, in <module>
    import fnmatch
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", lin

In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import numpy as np

In [2]:
s3_clickstream_path = "jobathon_click_data.json"
s3_login_path = "jobathon_login_data.csv"

In [3]:
# Import libraries
# Your code goes inside this function
# Function input - spark object, click data path, resolved data path
# Function output - final spark dataframe
def sample_function(spark, s3_clickstream_path, s3_login_path):
    df_clickstream =  spark.read.format("json").load(s3_clickstream_path)
    user_mapping =  spark.read.format("csv").option("header",True).load(s3_login_path)

    df_clickstream = df_clickstream.withColumn("current_page_url", col("client_side_data.current_page_url"))

    table = df_clickstream.join(user_mapping, (user_mapping["session_id"] == df_clickstream["session_id"])&(to_date(user_mapping["login_date_time"])== to_date(df_clickstream["event_date_time"])), "left").drop(user_mapping["session_id"])
    table = table.fillna("unregistered", subset=["user_id"])
    table = table.withColumn("current_date", substring(col("event_date_time"),1,10))
    table = table.withColumn("login_date", to_date("login_date_time"))
    table = table.withColumn("logged_in", when(table["user_id"] == "unregistered", 0).otherwise(1))
    table = table.withColumn("Click", when(table["event_type"] == "click", 1).otherwise(0))
    table = table.withColumn("Pageload", when(table["event_type"] == "pageload", 1).otherwise(0))

    url = table.groupBy("current_date", "user_id", "session_id", "browser_id").agg(min(to_timestamp("event_date_time")).alias("login"))
    url = url.join(table.select("event_date_time","current_page_url"), ((url["login"] == table["event_date_time"]) &(url["user_id"]== table["user_id"]) &(url["session_id"] == table["session_id"]) &(url["browser_id"]==table["browser_id"])), "left")
    url = url.withColumnRenamed("current_page_url", "first_url")
   
    table = table.join(url,((table["current_date"] == url["Current_date"])& (table["user_id"]== url["user_id"])&(url["session_id"]== table["session_id"])&(url["browser_id"]==table["browser_id"])), "left").drop(url["current_date"]).drop(url["user_id"]).drop(url["event_date_time"]).drop(url["session_id"]).drop(url["browser_id"]).drop(url["logged_in"])

    grouped = table.groupBy("current_date","browser_id","user_id","logged_in","first_url").agg(sum(table["Click"]).alias("number_of_clicks"), sum(table["Pageload"]).alias("number_of_pageloads"))
    # grouped = grouped.filter((grouped["number_of_clicks"] >0) | (grouped["user_id"] != "unregistered"))
    # grouped = grouped.join(url, ((url["current_date"] == grouped["current_date"]) & (url["user_id"] == grouped["user_id"])), "left").drop(url["current_date"]).drop(url["login"]).drop(url["event_date_time"]).drop(url["user_id"])
    # grouped = grouped.withColumnRenamed("current_page_url", "first_url")
    grouped = grouped.replace("unregistered",None, subset= ["user_id"])
    grouped = grouped.withColumn("current_date", col("current_date").cast("string")).withColumn("logged_in", col("logged_in").cast("string")).withColumn("number_of_clicks",col("number_of_clicks").cast("string")).withColumn("number_of_pageloads", col("number_of_pageloads").cast("string"))

    df_union = grouped.select("current_date","browser_id","user_id","logged_in","first_url","number_of_clicks","number_of_pageloads")
    
    df_union.createOrReplaceTempView("df_union_tbl")
    
    df_result = spark.sql("select * from df_union_tbl")
		
    return df_result

session = SparkSession.builder.appName("firstsql").master("local").getOrCreate()
sample_function(session, s3_clickstream_path, s3_login_path).count()

KeyboardInterrupt: 

In [ ]:
session = SparkSession.builder.appName("firstsql").master("local").getOrCreate()
readdata = session.read

In [ ]:
login_data = readdata.option("header", True).option("inferSchema", True).csv("jobathon_login_data.csv")

In [ ]:
click_data = readdata.option("header", True).option("inferSchema", True).json("jobathon_click_data.json")

In [ ]:
login_data.printSchema()

root
 |-- login_date_time: timestamp (nullable = true)
 |-- session_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [ ]:
click_data = click_data.withColumn("current_page_url", col("client_side_data.current_page_url"))
click_data = click_data.withColumn("time_elapsed", col("client_side_data.time_elapsed"))

In [ ]:
table = click_data.join(login_data, login_data["session_id"] == click_data["session_id"], "left").drop(login_data["session_id"])

In [ ]:
table = table.fillna("unregistered", subset=["user_id"])

In [ ]:
table = table.withColumn("Current_date", to_date("event_date_time"))
table = table.withColumn("login_date", to_date("login_date_time"))

In [ ]:
table = table.withColumn("logged_in", when(table["user_id"] == table["Current_date"], 1).otherwise(0))

In [ ]:
table = table.withColumn("Click", when(table["event_type"] == "click", 1).otherwise(0))
table = table.withColumn("Pageload", when(table["event_type"] == "pageload", 1).otherwise(0))

In [ ]:
url = table.groupBy("Current_date", "user_id","session_id", "browser_id").agg(min(to_timestamp("event_date_time")).alias("login"))
url.show()

+------------+------------+--------------------+--------------------+--------------------+
|Current_date|     user_id|          session_id|          browser_id|               login|
+------------+------------+--------------------+--------------------+--------------------+
|  2022-07-31|unregistered|EHDMvuQhE4H41ACxt...|Cms7xSIlU0pMBTMfR...|2022-07-31 04:30:...|
|  2022-07-31|unregistered|fM3e0bR1bsJaDssK3...|2N5Es6i1a1LuD4P1L...|2022-07-31 05:43:...|
|  2022-07-31|unregistered|Sgutf1dUl8dtPDTlw...|0ThrrpKVyk273B97M...|2022-07-31 09:17:...|
|  2022-07-31|unregistered|lKRh64hO1mAnK8bPH...|e39p9hf5aQW3ucqYs...|2022-07-31 10:39:...|
|  2022-07-31|unregistered|5XsVn8f03aU4pQQ8R...|OV97rA995d4edUEHm...|2022-07-31 02:59:...|
|  2022-07-31|unregistered|Lv2IuSFAoabPMEFgF...|xNZcJpYguHy7NWX2e...|2022-07-31 18:36:...|
|  2022-08-01|unregistered|wkJyZEAMXNKSqF4Sl...|jNhY7j8Z68RJt7Wtk...|2022-08-01 02:55:...|
|  2022-08-01|unregistered|FylH6umDqwegxEUJN...|xJDnFXsAegQb7jqL3...|2022-08-01 10:34:...|

In [ ]:
url.count()

305456

In [ ]:
url = url.join(table.select("event_date_time","current_page_url"), ((url["login"] == table["event_date_time"]) &(url["user_id"]== table["user_id"]) &(url["session_id"] == table["session_id"]) &(url["browser_id"]==table["browser_id"])), "left")

In [ ]:
url.show()

+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Current_date|     user_id|          session_id|          browser_id|               login|     event_date_time|    current_page_url|
+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  2022-07-31|unregistered|5XsVn8f03aU4pQQ8R...|OV97rA995d4edUEHm...|2022-07-31 02:59:...|2022-07-31 02:59:...|https://www.gosho...|
|  2022-07-31|unregistered|EHDMvuQhE4H41ACxt...|Cms7xSIlU0pMBTMfR...|2022-07-31 04:30:...|2022-07-31 04:30:...|https://www.gosho...|
|  2022-07-31|unregistered|fM3e0bR1bsJaDssK3...|2N5Es6i1a1LuD4P1L...|2022-07-31 05:43:...|2022-07-31 05:43:...|https://www.gosho...|
|  2022-07-31|unregistered|Sgutf1dUl8dtPDTlw...|0ThrrpKVyk273B97M...|2022-07-31 09:17:...|2022-07-31 09:17:...|https://www.gosho...|
|  2022-07-31|unregistered|lKRh64hO1mAnK8bPH...|e39p9hf5aQW3ucqYs...|

In [ ]:
url = url.withColumnRenamed("current_page_url", "first_url")

In [ ]:
table = table.join(url,((table["current_date"] == url["current_date"])& (table["user_id"]== url["user_id"])&(url["session_id"]== table["session_id"])&(url["browser_id"]==table["browser_id"])), "left").drop(url["current_date"]).drop(url["user_id"]).drop(url["event_date_time"]).drop(url["session_id"]).drop(url["browser_id"])

In [ ]:
table.show()

+--------------------+--------------------+----------+--------------------+------------+---------------+----------+---------+-----+--------+------------+------------+--------------------+--------------------+--------------------+--------------------+
|    client_side_data|     event_date_time|event_type|    current_page_url|time_elapsed|login_date_time|login_date|logged_in|Click|Pageload|Current_date|     user_id|          session_id|          browser_id|               login|           first_url|
+--------------------+--------------------+----------+--------------------+------------+---------------+----------+---------+-----+--------+------------+------------+--------------------+--------------------+--------------------+--------------------+
|{https://www.gosh...|2022-07-31 03:08:...|  pageload|https://www.gosho...|        null|           null|      null|        0|    0|       1|  2022-07-31|unregistered|7yQ7NY80yCBoyvoWf...|fmhMaS22p4KeDjH60...|2022-07-31 03:08:...|https://www.gosho.

In [ ]:
grouped = table.groupBy("current_date","browser_id","user_id","logged_in","first_url").agg(sum(table["Click"]).alias("number_of_clicks"), sum(table["Pageload"]).alias("number_of_pageloads"))

In [ ]:
grouped.printSchema()

root
 |-- current_date: date (nullable = true)
 |-- browser_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- logged_in: integer (nullable = false)
 |-- first_url: string (nullable = true)
 |-- number_of_clicks: long (nullable = true)
 |-- number_of_pageloads: long (nullable = true)



In [ ]:
grouped = grouped.withColumn("current_date", col("current_date").cast(StringType())).withColumn("logged_in", col("logged_in").cast(StringType())).withColumn("number_of_clicks",col("number_of_clicks").cast(StringType())).withColumn("number_of_pageloads", col("number_of_pageloads").cast(StringType()))

In [ ]:
grouped = grouped.filter(grouped["numbers_of_clicks"] > 0)

AnalysisException: ignored

In [ ]:
grouped.count()

In [ ]:
grouped = grouped.join(url, ((url["Current_date"] == grouped["Current_date"]) & (url["user_id"] == grouped["user_id"])), "left").drop(url["Current_date"]).drop(url["login"]).drop(url["event_date_time"]).drop(url["user_id"])

In [ ]:
grouped.count()

In [ ]:
grouped.show()

In [ ]:
grouped.select("Current_date", "browser_id", "user_id", "Login", "current_page_url").show()